In [38]:
# import packages
import pandas as pd
import numpy as np

from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

import time

In [27]:
# define name list to loop through test data sets
liste = ["test_50hz",
"test_amprion",
"test_tennet",
"test_transnet"]

In [49]:
# Run ARIMA rolling window forecast
for dframe in liste:
    print(dframe)
    df = pd.read_parquet(f"data/{dframe}.parquet")
    display(x.head(1))  
    startt =time.time()
    # Initialize relevant lists
    results = []
    iteration = 1

    # Sample data series
    CI_series = df["target"]
    CI_series.index = CI_series.index * np.timedelta64(1, 'h') + pd.Timestamp('2015-01-05 00:00:00+0100', tz='Europe/Berlin')
    CI_series.index.freq = 'h'

    # Define windows
    train_window_size = 28 * 24
    forecast_horizon = 24

    # Rolling window approach
    for start in range(0, len(CI_series) - train_window_size - forecast_horizon + 1, forecast_horizon):

        # Define the train and test windows
        train_window = CI_series[start:start + train_window_size]
        test_window = CI_series[start + train_window_size:start + train_window_size + forecast_horizon]

        # Use auto_arima to find the best ARIMA model
        pmd_arima = auto_arima(train_window, start_q=0, seasonal=False)
        model_order = pmd_arima.get_params().get("order")
        #print(CI_series.index[start + train_window_size],len(train_window), pmd_arima.get_params().get("order") )
        # Fit the ARIMA model
        model = ARIMA(train_window, order=model_order)
        model_fit = model.fit()

        # Forecast next 24 hours
        forecast = model_fit.forecast(steps=forecast_horizon)

        # Record results for each forecasted point
        for i, (fcst, actual) in enumerate(zip(forecast, test_window)):
            date = test_window.index[i]
            results.append({
                "date": date,
                "forecast": fcst,
                "actual": actual,
                "iteration": iteration,
                "start_train_window": train_window.index[0],
                "end_train_window": train_window.index[-1],
                "start_test_window": test_window.index[0],
                "end_test_window": test_window.index[-1]
            })

    # Create a DataFrame from the collected results
    results_df = pd.DataFrame(results)
    results_df.set_index("date", inplace=True)
    # Save forecasting results
    results_df.to_parquet(f"data/arima_results_{dframe}.parquet")




23


,target,Total_Emissions_in_kg,Total_Generation,Biomass,Fossil Brown coal/Lignite,Fossil Gas,Fossil Hard coal,Fossil Oil,Geothermal,Hydro Pumped Storage,...,share_ren,hour_of_day,day_of_month,day_of_year,month_of_year,week_of_year,day_of_week,quarter_of_year,time_idx,50HZ
71472,711.130926,40008937.0,56261.0,4135.0,31670.0,5260.0,4193.0,1140.0,0.0,79.0,...,0.210854,0,2,61,3,9,3,1,71472,50HZ


2023-03-30 01:00:00+02:00 672 (2, 1, 0)
2023-03-31 01:00:00+02:00 672 (2, 1, 0)
2023-04-01 01:00:00+02:00 672 (3, 1, 3)


/home/RDC/tramburc/.conda/envs/stock_tft/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


2023-04-02 01:00:00+02:00 672 (2, 1, 0)
2023-04-03 01:00:00+02:00 672 (2, 1, 0)
2023-04-04 01:00:00+02:00 672 (2, 1, 0)
2023-04-05 01:00:00+02:00 672 (2, 1, 0)
2023-04-06 01:00:00+02:00 672 (3, 0, 2)
2023-04-07 01:00:00+02:00 672 (2, 1, 2)


...

2024-02-24 00:00:00+01:00 672 (2, 1, 2)


/home/RDC/tramburc/.conda/envs/stock_tft/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/RDC/tramburc/.conda/envs/stock_tft/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


2024-02-25 00:00:00+01:00 672 (2, 1, 2)


/home/RDC/tramburc/.conda/envs/stock_tft/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/RDC/tramburc/.conda/envs/stock_tft/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


2024-02-26 00:00:00+01:00 672 (2, 1, 1)
2024-02-27 00:00:00+01:00 672 (4, 1, 1)
2024-02-28 00:00:00+01:00 672 (4, 1, 1)
2024-02-29 00:00:00+01:00 672 (2, 0, 1)
MAPE Value: 0.2604


In [50]:
result_50hz = pd.read_parquet("data/arima_results_test_50hz.parquet")
result_amprion = pd.read_parquet("data/arima_results_test_amprion.parquet")
result_tennet = pd.read_parquet("data/arima_results_test_tennet.parquet")
result_transnet = pd.read_parquet("data/arima_results_test_transnet.parquet")

